In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
acquisition1  = pd.read_csv('results/mouvement_exterieur_4/mouvement_exterieur_4_imu.csv')
acquisition2  = pd.read_csv('results/mouvement_exterieur_4/mouvement_exterieur_4_imu.csv')
acquisition3  = pd.read_csv('results/mouvement_exterieur_4/mouvement_exterieur_4_imu.csv')
acquisition4  = pd.read_csv('results/mouvement_exterieur_4/mouvement_exterieur_4_imu.csv')
acquisition5  = pd.read_csv('results/mouvement_exterieur_5/mouvement_exterieur_5_imu.csv')
acquisition6  = pd.read_csv('results/mouvement_exterieur_6/mouvement_exterieur_6_imu.csv')
acquisition7  = pd.read_csv('results/mouvement_exterieur_7/mouvement_exterieur_7_imu.csv')
acquisition8  = pd.read_csv('results/mouvement_exterieur_8/mouvement_exterieur_8_imu.csv')
acquisition9  = pd.read_csv('results/mouvement_exterieur_9/mouvement_exterieur_9_imu.csv')
acquisition10 = pd.read_csv('results/mouvement_exterieur_10/mouvement_exterieur_10_imu.csv')
acquisition11 = pd.read_csv('results/mouvement_exterieur_11/mouvement_exterieur_11_imu.csv')
acquisition12 = pd.read_csv('results/mouvement_exterieur_12/mouvement_exterieur_12_imu.csv')
acquisition13 = pd.read_csv('results/mouvement_exterieur_13/mouvement_exterieur_13_imu.csv')
acquisition14 = pd.read_csv('results/mouvement_exterieur_14/mouvement_exterieur_14_imu.csv')

acquisitions = [
    acquisition1,
    acquisition2,
    acquisition3,
    acquisition4,
    acquisition5,
    acquisition6,
    acquisition7,
    acquisition8,
    acquisition9,
    acquisition10,
    acquisition11,
    acquisition12,
    acquisition13,
    acquisition14
]

In [29]:
acquisition1

,Timestamp,Accel X,Accel Y,Accel Z,Gyro X,Gyro Y,Gyro Z,Magneto X,Magneto Y,Magneto Z
0,1.745335e+09,0.994483,0.065697,0.110044,-0.23625,0.34125,0.41125,-0.318328,0.128179,-0.176557
1,1.745335e+09,1.003694,0.076860,0.106323,-0.50750,0.28000,0.35000,-0.318328,0.128179,-0.176557
2,1.745335e+09,0.998021,0.064843,0.110593,-0.04375,0.29750,0.68250,-0.318036,0.126717,-0.180941
3,1.745335e+09,0.998387,0.062098,0.108702,1.56625,0.74375,0.93625,-0.318036,0.126717,-0.180941
4,1.745335e+09,0.995154,0.065514,0.105286,0.39375,-0.01750,0.88375,-0.319497,0.129933,-0.176703
...,...,...,...,...,...,...,...,...,...,...
50340,1.745336e+09,0.847351,0.038491,0.315370,34.92125,10.08875,-0.34125,-0.332797,0.092078,-0.199211
50341,1.745336e+09,0.871385,0.040565,0.267363,34.53625,11.08625,-1.61875,-0.329290,0.090909,-0.195703
50342,1.745336e+09,0.875411,0.053863,0.252052,34.09875,11.64625,-1.31250,-0.329290,0.090909,-0.195703
50343,1.745336e+09,0.870348,0.042456,0.256871,33.39000,13.07250,-0.14875,-0.327682,0.090909,-0.200526


# Test de Levene

https://datatab.fr/tutorial/levene-test



    H0: Les groupes ont des variances égales
    H1: Les groupes ont des variances différentes


In [30]:
import numpy as np
from scipy.stats import f



#La normalisation des données peut se faire avec la moyenne ou bien la mediane, 
#on aura décidé d'utiliser la medianne car c'est celle utilisé par scipy
def normalisation_moyenne(serie):
    moyenne = np.mean(serie)
    return [abs(x - moyenne) for x in serie]

def normalisation_median(serie):
    med = np.median(serie)
    return [abs(x - med) for x in serie]

def levene_a_la_main(acquisitions, type_acquisition):
    # Normaliser chaque groupe
    donnees_norm = []
    moyennes = []
    effectifs = []

    for acquisition in acquisitions:
        donnees_norm.append(normalisation_median(acquisition[type_acquisition]))
        moyennes.append(np.mean(normalisation_median(acquisition[type_acquisition])))
        effectifs.append(len(acquisition[type_acquisition]))

    # Moyenne globale de toutes les données normalisées regroupé
    toutes_donnees = [val for groupe in donnees_norm for val in groupe]
    moyenne_globale = np.mean(toutes_donnees)

    # SSB: Somme des carrés entre groupes
    SSB = 0
    for idx in range(len(acquisitions)):
        n_j = effectifs[idx]
        SSB += n_j * (moyennes[idx] - moyenne_globale) ** 2

    # SSW: Somme des carrés intra-groupes
    SSW = 0
    for idx in range(len(acquisitions)):
        values = donnees_norm[idx]
        for val in values:
            SSW += (val - moyennes[idx]) ** 2

    # Degrés de liberté
    k = len(acquisitions)
    N = sum(effectifs)
    dl1 = k - 1
    dl2 = N - k

    # Moyennes des carrés
    MSB = SSB / dl1
    MSW = SSW / dl2

    # Statistique F
    F_stat = MSB / MSW

    # Calcul de la p-value
    p_value = 1 - f.cdf(F_stat, dl1, dl2)
    
    return F_stat, p_value

In [31]:
stat, p_value = levene_a_la_main(acquisitions, "Accel X")
print(f"Statistique de test (fait main) : {stat:.4f}")
print(f"P-value : {p_value:.4f}")

Statistique de test (fait main) : 1324.5121
P-value : 0.0000


In [32]:
#Verification
from scipy.stats import levene

stat2, p_value2 = levene(
    acquisitions[0]["Accel X"],
    acquisitions[1]["Accel X"],
    acquisitions[2]["Accel X"],
    acquisitions[3]["Accel X"],
    acquisitions[4]["Accel X"],
    acquisitions[5]["Accel X"],
    acquisitions[6]["Accel X"],
    acquisitions[7]["Accel X"],
    acquisitions[8]["Accel X"],
    acquisitions[9]["Accel X"],
    acquisitions[10]["Accel X"],
    acquisitions[11]["Accel X"],
    acquisitions[12]["Accel X"],
    acquisitions[13]["Accel X"]
)

print(f"Statistique (Scipy) : {stat2}")
print(f"P-value : {p_value2}")

Statistique (Scipy) : 1324.512096402796
P-value : 0.0


#### Quel paramétrage ?

Comme on sait qu'il y a bien une différence parmi les acquisitions due au paramétrage.


# Quel paramétrage ?

Comme on sait qu'il y a bien une différence parmi les acquisitions due au paramétrage.


In [34]:
colonnes = acquisitions[0].columns[1:]

for col in colonnes:
    stds = []
    for acq in acquisitions:
        series = acq[col]
        stds.append(np.std(series))     
        
    idx_sorted = np.argsort(stds)

    print(f"Colonne {col} :")
    for rang, idx in enumerate(idx_sorted[:3], start=1):
        print(f"  {rang}. Acquisition {idx+1}, écart-type = {stds[idx]:.5f}")
    print(" ")  


Colonne Accel X :
  1. Acquisition 11, écart-type = 0.20029
  2. Acquisition 9, écart-type = 0.20898
  3. Acquisition 10, écart-type = 0.21107
 
Colonne Accel Y :
  1. Acquisition 13, écart-type = 0.06956
  2. Acquisition 11, écart-type = 0.07087
  3. Acquisition 10, écart-type = 0.07218
 
Colonne Accel Z :
  1. Acquisition 13, écart-type = 0.09181
  2. Acquisition 8, écart-type = 0.09538
  3. Acquisition 11, écart-type = 0.10488
 
Colonne Gyro X :
  1. Acquisition 1, écart-type = 17.06821
  2. Acquisition 2, écart-type = 17.06821
  3. Acquisition 3, écart-type = 17.06821
 
Colonne Gyro Y :
  1. Acquisition 6, écart-type = 11.86876
  2. Acquisition 1, écart-type = 12.49586
  3. Acquisition 2, écart-type = 12.49586
 
Colonne Gyro Z :
  1. Acquisition 10, écart-type = 10.57794
  2. Acquisition 13, écart-type = 10.90263
  3. Acquisition 8, écart-type = 11.53176
 
Colonne Magneto X :
  1. Acquisition 8, écart-type = 0.02630
  2. Acquisition 5, écart-type = 0.02869
  3. Acquisition 12, écar